In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from time import sleep
from tqdm import tqdm

In [2]:
foldername = 'ffABC1'
n_inds = 30
scale = "60"

angle_thresh = 30
time_thresh = 5
thresh_folder = 'angle' + str(angle_thresh) + '_time' + str(time_thresh)

In [3]:
ind = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + thresh_folder + '/individual_' + scale + '.csv')
ind.drop(['Unnamed: 0'], axis=1, inplace=True)

pair = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + thresh_folder + '/pairwise_' + scale + '.csv')
pair.drop(['Unnamed: 0'], axis=1, inplace=True)
pair.head()

,f_id,n_id,dist,speed_diff,acc_diff,#t(centisec),tau,cc,ang_pos
0,0,1,6.857518,1.198974,0.418019,300,2,0.0,2.683285
1,0,2,3.755912,-0.728026,-0.380283,300,0,0.0,0.218072
2,0,3,4.044581,-0.364281,-0.429167,300,0,0.0,2.139774
3,0,4,4.507002,-0.580222,-0.360962,300,0,0.0,3.076962
4,0,5,3.997078,-0.028310,-0.040836,300,0,0.0,0.082241


In [4]:
assert(np.min(ind['#t(centisec)']) == np.min(pair['#t(centisec)'])) 
assert(np.max(ind['#t(centisec)']) == np.max(pair['#t(centisec)']))

In [5]:
df_ind = pair.loc[:,['f_id', '#t(centisec)']].drop_duplicates()
df_ind['reach'] = 0

for fr in tqdm(np.unique(pair['#t(centisec)'])):
    tmp = pair[(pair['#t(centisec)'] == fr) & (pair['cc'] == 1)].reset_index()
    
    G = nx.DiGraph()
    G.add_nodes_from(np.unique(pair['f_id']))
    
    list_a = list(tmp.loc[tmp['cc'] == 1, 'n_id'].values) 
    list_b = list(tmp.loc[tmp['cc'] == 1, 'f_id'].values)
    
    if len(list_a) != 0:
        G.add_edges_from(list(zip(list_a, list_b)))

        for idx in np.unique(pair['n_id']):
            df_ind.loc[(pair['#t(centisec)'] == fr) & (pair['f_id'] == idx), 'reach'] = nx.local_reaching_centrality(G, idx)
        
    sleep(0.001)
    
df = pd.merge(ind,df_ind)

100%|███████████████████████████████████| 11481/11481 [2:51:55<00:00,  1.11it/s]


In [6]:
df.to_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + thresh_folder + '/individual_' + scale + '.csv', mode='w')